In [272]:
import json
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', None)

In [273]:
pro = json.load(open("./task3_product.json"))

In [274]:
idx2label = json.load(open("./idx2label.json"))
label2idx = dict(zip(idx2label.values(), idx2label.keys()))
label2lv1 = json.load(open("./label2level1.json")) # lv5 index에 대응되는 lv1 keyward
label2lv2 = json.load(open("./label2level2.json"))
lv1toidx = json.load(open("./lv1toidx.json"))
lv2toidx = json.load(open("./lv2toidx.json")) # level keyword를 순서대로 indexing
lv4toidx = json.load(open("./lv4toidx.json"))
zodallv1 = json.load(open("./zodal_lv1.json"))
zodallv2 = json.load(open("./zodal_lv2.json"))

In [275]:
lv1tolv5idx = dict(zip(label2lv1.values(), label2lv1.keys()))
lv2tolv5idx = dict(zip(label2lv2.values(), label2lv2.keys()))

In [276]:
lv1toprompt = {}
for t in zodallv1:
    tmp = t['input'].split('입니다.')[0].split('다음 물품 분류는 ')[-1]
    lv1toprompt[tmp] = t['input']

In [277]:
lv2toprompt = {}
for t in zodallv2:
    tmp = t['input'].split('입니다.')[0].split('다음 물품 분류는 ')[-1]
    lv2toprompt[tmp] = t['input']

In [278]:
label2lv1['21834']

'융복합상품'

In [279]:
from collections import defaultdict
lv1box = defaultdict(list)
lv2box = defaultdict(list)

for k, v in label2lv1.items():
    lv1box[v].append(int(k))

for k, v in label2lv2.items():
    lv2box[v].append(int(k))

max_l1 = 0
for k, v in lv1box.items():
    if len(v) > max_l1:
        max_l1 = len(v)

max_l2 = 0
for k, v in lv2box.items():
    if len(v) > max_l2:
        max_l2 = len(v)

lv1box_np = -1*np.ones((len(lv1box), max_l1))
lv1box_np = lv1box_np.astype(int)

lv2box_np = -1*np.ones((len(lv2box), max_l2))
lv2box_np = lv2box_np.astype(int)

for i, (k, v) in enumerate(lv1box.items()):
    lv1box_np[i,:len(v)] = np.array(v).astype(int)

for i, (k, v) in enumerate(lv2box.items()):
    lv2box_np[i,:len(v)] = np.array(v).astype(int)

tmp1 = []
for k, v in lv1box.items():
    tmp1.append(len(v))

tmp2 = []
for k, v in lv2box.items():
    tmp2.append(len(v))

In [280]:
np.where(lv1box_np==22251)

(array([], dtype=int64), array([], dtype=int64))

In [281]:
lv1box_np

array([[    0,     1,     2, ...,    -1,    -1,    -1],
       [  176,   177,   178, ...,    -1,    -1,    -1],
       [  400,   401,   402, ...,    -1,    -1,    -1],
       ...,
       [12269, 12270, 12271, ...,    -1,    -1,    -1],
       [12272, 12273, 12274, ...,    -1,    -1,    -1],
       [12277, 12278, 12279, ..., 21832, 21833, 21834]])

In [282]:
# np.where(lv1box_np == -1, 0, 1).sum(axis=1) == np.array(tmp1)

In [283]:
# np.where(lv2box_np == -1, 0, 1).sum(axis=1) == np.array(tmp2)

In [284]:
lv1box_np

array([[    0,     1,     2, ...,    -1,    -1,    -1],
       [  176,   177,   178, ...,    -1,    -1,    -1],
       [  400,   401,   402, ...,    -1,    -1,    -1],
       ...,
       [12269, 12270, 12271, ...,    -1,    -1,    -1],
       [12272, 12273, 12274, ...,    -1,    -1,    -1],
       [12277, 12278, 12279, ..., 21832, 21833, 21834]])

In [285]:
import torch

In [286]:
lv1box_emb = torch.nn.Embedding(lv1box_np.shape[0], lv1box_np.shape[1])
lv2box_emb = torch.nn.Embedding(lv2box_np.shape[0], lv2box_np.shape[1])

In [287]:
lv1box_emb.requires_grad_(False)
lv2box_emb.requires_grad_(False)

Embedding(387, 983)

In [288]:
lv1box_tensor = torch.tensor(lv1box_np, dtype=torch.long)
lv2box_tensor = torch.tensor(lv2box_np, dtype=torch.long)

In [289]:
lv1box_emb.weight.data.copy_(lv1box_tensor.int())
lv2box_emb.weight.data.copy_(lv2box_tensor.int())

tensor([[ 0.0000e+00,  1.0000e+00,  2.0000e+00,  ..., -1.0000e+00,
         -1.0000e+00, -1.0000e+00],
        [ 2.2000e+01,  2.3000e+01,  2.4000e+01,  ..., -1.0000e+00,
         -1.0000e+00, -1.0000e+00],
        [ 2.5000e+01,  2.6000e+01,  2.7000e+01,  ..., -1.0000e+00,
         -1.0000e+00, -1.0000e+00],
        ...,
        [ 2.1565e+04,  2.1566e+04,  2.1567e+04,  ..., -1.0000e+00,
         -1.0000e+00, -1.0000e+00],
        [ 2.1653e+04,  2.1654e+04,  2.1655e+04,  ..., -1.0000e+00,
         -1.0000e+00, -1.0000e+00],
        [ 2.1797e+04,  2.1798e+04,  2.1799e+04,  ..., -1.0000e+00,
         -1.0000e+00, -1.0000e+00]])

In [18]:
# torch.save(lv1box_emb, 'lv1box_emb.pt')
# torch.save(lv2box_emb, 'lv2box_emb.pt')

### Todo
- idx, pro, cat, lv1, lv2, pro_cat_lv1, zodal_cat_lv1, pro_cat_lv2, zodal_cat_lv2 columns의 dataframe 만들기

In [19]:
df = pd.DataFrame(columns=['pro', 'cat', 'lv1', 'zodal_cat_lv1', 'lv2', 'zodal_cat_lv2', 'lv1label', 'lv2label'])

In [20]:
pro_list, cat_list, lv1_list, lv2_list, zodal_cat_lv1_list, zodal_cat_lv2_list, lv1label_list, lv2label_list = [], [], [], [], [], [], [], []
for i, d in enumerate(pro):
    pro_list.append(d['input'])
    cat_list.append(d['output'])
    label = d['output'].split(",")[0].split("다음 물품 분류는 ")[-1]
    lv1_list.append(label2lv1[label2idx[label]])
    lv2_list.append(label2lv2[label2idx[label]])
    zodal_cat_lv1_list.append(lv1toprompt[label2lv1[label2idx[label]]])
    zodal_cat_lv2_list.append(lv2toprompt[label2lv2[label2idx[label]]])
    lv1label_list.append(lv1toidx[label2lv1[label2idx[label]]])
    lv2label_list.append(lv2toidx[label2lv2[label2idx[label]]])

In [29]:
lv1toprompt.keys()

dict_keys(['산동식물및동식물성생산품', '광물,직물및비식용동식물자원', '화학제품', '수지,고무,탄성중합체', '종이원료및종이제품', '연료,연료첨가제,윤활유및방부식제', '광산기계및액세서리', '농.수.임.축산용기계', '건축건설기계및보조용품', '산업용제조가공기계및액세서리', '물품취급,조정,저장기계,액세서리및소모품', '상용,군용,개인용운송기구및액세서리와부품', '회전기기및경전기', '공구및범용기계', '건자재', '제조부품', '전자부품및소모품', '전기시스템,조명,부품,액세서리및보조용품', '배관유체조절시스템장비및부품', '실험실용실험,측정,관측및검사기기', '의료용기기', '정보기술방송및통신기', '사무용기기액세서리및용품', '인쇄,사진및시청각기기', '공공안전및치안장비', '위생장비및용품', '서비스업용기계장비및용품', '스포츠및레크리에이션장비용품및액세서리', '식음료품및담배제품', '의약품', '가정용품및가전제품', '의류,가방및개인관리용품', '시계,보석및원석제품', '출판물', '가구및관련제품', '악기,게임,장난감,미술작품,공예품,교육용장비,교재,교육용품및교육용보조품', '농업,어업,임업및야생동물관련서비스', '광업,석유및가스서비스', '건물및시설물건설유지보수서비스', '공산품제조서비스', '산업위생관련서비스', '환경관련서비스', '운송과보관및우편관련서비스', '경영관련서비스', '공학연구및기술기반서비스', '편집디자인그래픽및예술관련서비스', '공공사업및공공부문관련서비스', '금융및보험서비스', '보건서비스', '교육및훈련서비스', '여행,음식,숙박및오락관련서비스', '개인및가정관련서비스', '국방및치안관련서비스', '정치및행정서비스', '조직및동호회', '융복합상품'])

In [35]:
list(lv1toidx.keys())

['산동식물및동식물성생산품',
 '광물,직물및비식용동식물자원',
 '화학제품',
 '수지,고무,탄성중합체',
 '종이원료및종이제품',
 '연료,연료첨가제,윤활유및방부식제',
 '광산기계및액세서리',
 '농.수.임.축산용기계',
 '건축건설기계및보조용품',
 '산업용제조가공기계및액세서리',
 '물품취급,조정,저장기계,액세서리및소모품',
 '상용,군용,개인용운송기구및액세서리와부품',
 '회전기기및경전기',
 '공구및범용기계',
 '건자재',
 '제조부품',
 '전자부품및소모품',
 '전기시스템,조명,부품,액세서리및보조용품',
 '배관유체조절시스템장비및부품',
 '실험실용실험,측정,관측및검사기기',
 '의료용기기',
 '정보기술방송및통신기',
 '사무용기기액세서리및용품',
 '인쇄,사진및시청각기기',
 '공공안전및치안장비',
 '위생장비및용품',
 '서비스업용기계장비및용품',
 '스포츠및레크리에이션장비용품및액세서리',
 '식음료품및담배제품',
 '의약품',
 '가정용품및가전제품',
 '의류,가방및개인관리용품',
 '시계,보석및원석제품',
 '출판물',
 '가구및관련제품',
 '악기,게임,장난감,미술작품,공예품,교육용장비,교재,교육용품및교육용보조품',
 '농업,어업,임업및야생동물관련서비스',
 '건물및시설물건설유지보수서비스',
 '공산품제조서비스',
 '산업위생관련서비스',
 '환경관련서비스',
 '운송과보관및우편관련서비스',
 '경영관련서비스',
 '공학연구및기술기반서비스',
 '편집디자인그래픽및예술관련서비스',
 '공공사업및공공부문관련서비스',
 '금융및보험서비스',
 '보건서비스',
 '교육및훈련서비스',
 '여행,음식,숙박및오락관련서비스',
 '개인및가정관련서비스',
 '국방및치안관련서비스',
 '정치및행정서비스',
 '융복합상품']

In [21]:
df['pro']=pro_list
df['cat']=cat_list
df['lv1']=lv1_list
df['lv2']=lv2_list
df['zodal_cat_lv1']=zodal_cat_lv1_list
df['zodal_cat_lv2']=zodal_cat_lv2_list
df['lv1label']=lv1label_list
df['lv2label']=lv2label_list

In [105]:
# file_name = "zodal_dataframe.xlsx"
# df.to_excel(file_name, engine='openpyxl', index=False)

In [27]:
df

,pro,cat,lv1,zodal_cat_lv1,lv2,zodal_cat_lv2,lv1label,lv2label
0,"이 상품의 이름은 로엘 듀스핀5 PRO이고, 해당 상품의 상세 설명은 제조회사 : ...","다음 물품 분류는 습식바닥청소기, 영어로 Floor scrubbers 입니다. 습식...",위생장비및용품,다음 물품 분류는 위생장비및용품입니다. 위생장비및용품의 상세설명은 다음과 같습니다....,산업용청소장비,다음 물품 분류는 산업용청소장비입니다. 산업용청소장비의 상세설명은 다음과 같습니다....,25,177
1,"이 상품의 이름은 한경희생활과학 마이스터 AM-7500 (블랙골드, AM-7500B...","다음 물품 분류는 습식바닥청소기, 영어로 Floor scrubbers 입니다. 습식...",위생장비및용품,다음 물품 분류는 위생장비및용품입니다. 위생장비및용품의 상세설명은 다음과 같습니다....,산업용청소장비,다음 물품 분류는 산업용청소장비입니다. 산업용청소장비의 상세설명은 다음과 같습니다....,25,177
2,"이 상품의 이름은 휴스톰 듀얼스핀 HS-10500이고, 해당 상품의 상세 설명은 제...","다음 물품 분류는 습식바닥청소기, 영어로 Floor scrubbers 입니다. 습식...",위생장비및용품,다음 물품 분류는 위생장비및용품입니다. 위생장비및용품의 상세설명은 다음과 같습니다....,산업용청소장비,다음 물품 분류는 산업용청소장비입니다. 산업용청소장비의 상세설명은 다음과 같습니다....,25,177
3,"이 상품의 이름은 신일전자 SDC-W8000NK이고, 해당 상품의 상세 설명은 제조...","다음 물품 분류는 습식바닥청소기, 영어로 Floor scrubbers 입니다. 습식...",위생장비및용품,다음 물품 분류는 위생장비및용품입니다. 위생장비및용품의 상세설명은 다음과 같습니다....,산업용청소장비,다음 물품 분류는 산업용청소장비입니다. 산업용청소장비의 상세설명은 다음과 같습니다....,25,177
4,"이 상품의 이름은 무궁화전자 바로바로 MDC-900이고, 해당 상품의 상세 설명은 ...","다음 물품 분류는 습식바닥청소기, 영어로 Floor scrubbers 입니다. 습식...",위생장비및용품,다음 물품 분류는 위생장비및용품입니다. 위생장비및용품의 상세설명은 다음과 같습니다....,산업용청소장비,다음 물품 분류는 산업용청소장비입니다. 산업용청소장비의 상세설명은 다음과 같습니다....,25,177
5,"이 상품의 이름은 경성오토비스 KAC-8000이고, 해당 상품의 상세 설명은 제조회...","다음 물품 분류는 습식바닥청소기, 영어로 Floor scrubbers 입니다. 습식...",위생장비및용품,다음 물품 분류는 위생장비및용품입니다. 위생장비및용품의 상세설명은 다음과 같습니다....,산업용청소장비,다음 물품 분류는 산업용청소장비입니다. 산업용청소장비의 상세설명은 다음과 같습니다....,25,177
6,"이 상품의 이름은 유닉스 UN-A1958이고, 해당 상품의 상세 설명은 제조회사 :...","다음 물품 분류는 헤어드라이어, 영어로 Domestic hair dryers 입니다...",가정용품및가전제품,다음 물품 분류는 가정용품및가전제품입니다. 가정용품및가전제품의 상세설명은 다음과 같...,가전기기,"다음 물품 분류는 가전기기입니다. 가전기기의 상세설명은 다음과 같습니다. 주방용, ...",30,222
7,"이 상품의 이름은 유닉스 슈퍼디플러스 UN-B6011이고, 해당 상품의 상세 설명은...","다음 물품 분류는 헤어드라이어, 영어로 Domestic hair dryers 입니다...",가정용품및가전제품,다음 물품 분류는 가정용품및가전제품입니다. 가정용품및가전제품의 상세설명은 다음과 같...,가전기기,"다음 물품 분류는 가전기기입니다. 가전기기의 상세설명은 다음과 같습니다. 주방용, ...",30,222
8,"이 상품의 이름은 유닉스 슈퍼디플러스 탭 UN-A6050이고, 해당 상품의 상세 설...","다음 물품 분류는 헤어드라이어, 영어로 Domestic hair dryers 입니다...",가정용품및가전제품,다음 물품 분류는 가정용품및가전제품입니다. 가정용품및가전제품의 상세설명은 다음과 같...,가전기기,"다음 물품 분류는 가전기기입니다. 가전기기의 상세설명은 다음과 같습니다. 주방용, ...",30,222
9,"이 상품의 이름은 다이슨 슈퍼소닉 HD01 (아이언/핑크)이고, 해당 상품의 상세 ...","다음 물품 분류는 헤어드라이어, 영어로 Domestic hair dryers 입니다...",가정용품및가전제품,다음 물품 분류는 가정용품및가전제품입니다. 가정용품및가전제품의 상세설명은 다음과 같...,가전기기,"다음 물품 분류는 가전기기입니다. 가전기기의 상세설명은 다음과 같습니다. 주방용, ...",30,222


In [301]:
zoraw = pd.read_csv("zodal_raw.csv")

In [302]:
zoraw.head(2)

,Unnamed: 0,context,response
0,13605,"물품분류번호는1010150201 이고, 분류체계는 개입니다",대분류: 산동식물및동식물성생산품 중분류: 산동물 소분류: 가축류 품명: 개
1,13606,"물품분류번호는1010150401 이고, 분류체계는 밍크입니다",대분류: 산동식물및동식물성생산품 중분류: 산동물 소분류: 가축류 품명: 밍크


In [315]:
lv1tolv2, lv1idxtolv2idx = defaultdict(list), defaultdict(list)
lv2tolv5 = {}
for i, row in zoraw.iterrows():
    gt_l1_cat = row['response'].split("대분류: ")[-1].split(" 중분류:")[0]
    gt_l2_cat = row['response'].split("중분류: ")[-1].split(" 소분류:")[0]
    lv1tolv2[gt_l1_cat].append(lv2toidx[gt_l2_cat])

    lv1idxtolv2idx[lv1toidx[gt_l1_cat]].append(lv2toidx[gt_l2_cat])

In [316]:
lv1idxtolv2idx

defaultdict(list,
            {0: [0,
              0,
              0,
              0,
              0,
              0,
              0,
              0,
              0,
              0,
              0,
              0,
              0,
              0,
              0,
              0,
              0,
              0,
              0,
              0,
              0,
              0,
              1,
              1,
              1,
              2,
              2,
              2,
              2,
              2,
              3,
              3,
              3,
              3,
              4,
              4,
              4,
              5,
              5,
              5,
              5,
              5,
              5,
              5,
              5,
              5,
              5,
              5,
              5,
              5,
              5,
              5,
              5,
              5,
              5,
              5,
              5,
          

In [312]:
lv1tolv2

{'산동식물및동식물성생산품': '해충및소형동물방제제품',
 '광물,직물및비식용동식물자원': '금속폐기물',
 '화학제품': '화합물및혼합물',
 '수지,고무,탄성중합체': '수지및로진',
 '종이원료및종이제품': '산업용지',
 '연료,연료첨가제,윤활유및방부식제': '윤활제,유류,그리스및방부식제',
 '광산기계및액세서리': '유류가스공사및생산장비',
 '농.수.임.축산용기계': '낚시및양식용장비',
 '건축건설기계및보조용품': '건설기계및중장비',
 '산업용제조가공기계및액세서리': '특수산업기기및장비',
 '물품취급,조정,저장기계,액세서리및소모품': '포장보조용품',
 '상용,군용,개인용운송기구및액세서리와부품': '철도용경보및신호장비',
 '회전기기및경전기': '원자및핵에너지기기',
 '공구및범용기계': '자동차용특수용구',
 '건자재': '구조재료',
 '제조부품': '자석및자성재료',
 '전자부품및소모품': '전자관장치및액세서리',
 '전기시스템,조명,부품,액세서리및보조용품': '전선관리장치액세서리및용품',
 '배관유체조절시스템장비및부품': '관류및관이음자재',
 '실험실용실험,측정,관측및검사기기': '실험실습기자재',
 '의료용기기': '정형외과수술용임플란트',
 '정보기술방송및통신기': '소프트웨어',
 '사무용기기액세서리및용품': '사무용품',
 '인쇄,사진및시청각기기': '사진제작용품',
 '공공안전및치안장비': '특수교육용장비및부속품',
 '위생장비및용품': '청소및위생용품',
 '서비스업용기계장비및용품': '장례용품',
 '스포츠및레크리에이션장비용품및액세서리': '운동장및놀이터시설물',
 '식음료품및담배제품': '통조림또는발효식품',
 '의약품': '마약',
 '가정용품및가전제품': '가정용실내벽걸이용품',
 '의류,가방및개인관리용품': '바느질용품',
 '시계,보석및원석제품': '시계류',
 '출판물': '표지용품및보조품',
 '가구및관련제품': '머천다이징용가구및액세서리',
 '악기,게임,장난감,미술작품,공예품,교육용장비,교재,교육용품및교육용